# import library

In [60]:
import pandas as pd
pd.set_option('display.max_columns', 100)

import requests
import json

import re

from pprint import pprint

# fips and label lists

## get fips list of all counties in CA

In [2]:
fips_ep_url = "https://www2.census.gov/geo/docs/reference/codes/files/st06_ca_cou.txt"

fips_txt = requests.get(fips_ep_url).text

fips_lines = fips_txt.split("\r\n")
fips_lines[:5]

fips_ls = []

for line in fips_lines:
    fips_ls.append(line.split(","))
    
fips_ls[:5]

fips_df = pd.DataFrame(fips_ls)
fips_df.drop([0, 1, 4], inplace=True, axis = 1)
fips_df.columns = ["fips", "county"]
fips_df.head()

,fips,county
0,001,Alameda County
1,003,Alpine County
2,005,Amador County
3,007,Butte County
4,009,Calaveras County


## get list of label id and names

In [3]:
label_ep_url = "https://api.census.gov/data/2017/acs/acs5/variables"
label_txt = requests.get(label_ep_url).text
label_ls = json.loads(label_txt)

label_dict = {}

for row in label_ls:
    label_dict[row[0]] = row[1]

label_df = pd.DataFrame(label_ls[1:], columns = label_ls[0])
label_df.head()

,name,label,concept
0,for,Census API FIPS 'for' clause,Census API Geography Specification
1,in,Census API FIPS 'in' clause,Census API Geography Specification
2,ucgid,Uniform Census Geography Identifier clause,Census API Geography Specification
3,B06004HPR_002E,Estimate!!Total!!Born in Puerto Rico,"PLACE OF BIRTH (WHITE ALONE, NOT HISPANIC OR L..."
4,B02019_008E,Estimate!!Total Groups Tallied!!Micronesian!!O...,NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALO...


# census  

## get block group data from us census API

In [4]:
c_fips_ls = list(fips_df["fips"])

cs_ls = []

for c_fips in c_fips_ls[:1]:

    cs_ep_url = "https://api.census.gov/data/2017/acs/acs5?" \
             "get=group(B05001),NAME&" \
             "for=block%20group:*&in=state:06+in=county:" + c_fips + "&" \
             "key=9e518d813c34cefe46bf0317ca1b54e57d30219a"

    cs_txt = requests.get(cs_ep_url).text
    cs_js = json.loads(cs_txt)
    
    cs_ls.extend(cs_js)
    
    print("fips {} : {}".format(c_fips, len(cs_ls)))

cs_df = pd.DataFrame(cs_ls[1:], columns = cs_ls[0])
cs_df.head()

fips 001 : 1048


,NAME,GEO_ID,B05001_001E,B05001_001M,B05001_002E,B05001_002M,B05001_003E,B05001_003M,B05001_004E,B05001_004M,B05001_005E,B05001_005M,B05001_006E,B05001_006M,NAME,B05001_001EA,B05001_001MA,B05001_002EA,B05001_002MA,B05001_003MA,B05001_003EA,B05001_004EA,B05001_004MA,B05001_005MA,B05001_005EA,B05001_006EA,B05001_006MA,state,county,tract,block group
0,"Block Group 1, Census Tract 4403.36, Alameda C...",1500000US060014403361,None,None,None,None,None,None,None,None,None,None,None,None,"Block Group 1, Census Tract 4403.36, Alameda C...",None,None,None,None,None,None,None,None,None,None,None,None,06,001,440336,1
1,"Block Group 2, Census Tract 4403.36, Alameda C...",1500000US060014403362,None,None,None,None,None,None,None,None,None,None,None,None,"Block Group 2, Census Tract 4403.36, Alameda C...",None,None,None,None,None,None,None,None,None,None,None,None,06,001,440336,2
2,"Block Group 1, Census Tract 4419.27, Alameda C...",1500000US060014419271,None,None,None,None,None,None,None,None,None,None,None,None,"Block Group 1, Census Tract 4419.27, Alameda C...",None,None,None,None,None,None,None,None,None,None,None,None,06,001,441927,1
3,"Block Group 2, Census Tract 4419.27, Alameda C...",1500000US060014419272,None,None,None,None,None,None,None,None,None,None,None,None,"Block Group 2, Census Tract 4419.27, Alameda C...",None,None,None,None,None,None,None,None,None,None,None,None,06,001,441927,2
4,"Block Group 1, Census Tract 4419.25, Alameda C...",1500000US060014419251,None,None,None,None,None,None,None,None,None,None,None,None,"Block Group 1, Census Tract 4419.25, Alameda C...",None,None,None,None,None,None,None,None,None,None,None,None,06,001,441925,1


## get tract data from us census API

In [5]:
cs_ls = []

cs_ep_url = "https://api.census.gov/data/2017/acs/acs5?" \
             "get=group(B05001)&" \
             "for=tract:*&in=state:06+in=county:*&" \
             "key=9e518d813c34cefe46bf0317ca1b54e57d30219a"

cs_txt = requests.get(cs_ep_url).text
cs_js = json.loads(cs_txt)
    
cs_ls.extend(cs_js)

cs_df = pd.DataFrame(cs_ls[1:], columns = cs_ls[0])
cs_df.dropna(axis = 1, inplace=True)
cs_cols_to_drop = [col for col in list(cs_df.columns) if re.match(".*M$", col)]
cs_df.drop(labels = cs_cols_to_drop, inplace=True, axis = 1)
cs_df.head()

,GEO_ID,B05001_001E,B05001_002E,B05001_003E,B05001_004E,B05001_005E,B05001_006E,NAME,state,county,tract
0,1400000US06001400100,2991,2033,11,46,653,248,"Census Tract 4001, Alameda County, California",06,001,400100
1,1400000US06001400200,1997,1668,11,38,158,122,"Census Tract 4002, Alameda County, California",06,001,400200
2,1400000US06001400300,5123,4241,13,78,539,252,"Census Tract 4003, Alameda County, California",06,001,400300
3,1400000US06001400400,3991,3336,0,79,301,275,"Census Tract 4004, Alameda County, California",06,001,400400
4,1400000US06001400500,3944,3258,0,111,228,347,"Census Tract 4005, Alameda County, California",06,001,400500


In [6]:
cs_cols = cs_df.columns

cs_cols_conv = []

for col in cs_cols:
    try:
        cs_cols_conv.append(label_dict[col])
    except:
        cs_cols_conv.append(col)

cs_df.columns = cs_cols_conv

cs_df.head()

,Geography,Estimate!!Total,Estimate!!Total!!U.S. citizen born in the United States,Estimate!!Total!!U.S. citizen born in Puerto Rico or U.S. Island Areas,Estimate!!Total!!U.S. citizen born abroad of American parent(s),Estimate!!Total!!U.S. citizen by naturalization,Estimate!!Total!!Not a U.S. citizen,NAME,state,county,tract
0,1400000US06001400100,2991,2033,11,46,653,248,"Census Tract 4001, Alameda County, California",06,001,400100
1,1400000US06001400200,1997,1668,11,38,158,122,"Census Tract 4002, Alameda County, California",06,001,400200
2,1400000US06001400300,5123,4241,13,78,539,252,"Census Tract 4003, Alameda County, California",06,001,400300
3,1400000US06001400400,3991,3336,0,79,301,275,"Census Tract 4004, Alameda County, California",06,001,400400
4,1400000US06001400500,3944,3258,0,111,228,347,"Census Tract 4005, Alameda County, California",06,001,400500


In [7]:
cs_df.to_csv("data/acs2017_tract.csv")